In [300]:
import os
import numpy as np
import pandas as pd
import altair as alt
import sys
import json

sys.path.insert(0, '..')
from agents import PsAgent
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [301]:
path = '../data/models/colision_0005_ref_1'

In [302]:
lista = [i for i in os.listdir(f'{path}') if i not in ['.ipynb_checkpoints']]

In [303]:
lista

['2025-03-27_18-47-02.622236__5',
 '2025-03-27_18-47-02.682592__18',
 '2025-03-27_18-47-02.774059__29',
 '2025-03-27_18-47-02.626591__8',
 '2025-03-27_18-47-02.746650__27',
 '2025-03-27_18-47-02.762682__28',
 '2025-03-27_18-47-02.671299__17',
 '2025-03-27_18-47-02.654623__15',
 '2025-03-27_18-47-02.670756__16',
 '2025-03-27_18-47-02.622753__6',
 '2025-03-27_18-47-02.623354__7',
 '2025-03-27_18-47-02.777751__31',
 '2025-03-27_18-47-02.620178__1',
 '2025-03-27_18-47-02.774548__30',
 '2025-03-27_18-47-02.710514__22',
 '2025-03-27_18-47-02.643225__14',
 '2025-03-27_18-47-02.621732__4',
 '2025-03-27_18-47-02.630939__12',
 '2025-03-27_18-47-02.621166__3',
 '2025-03-27_18-47-02.627046__9',
 '2025-03-27_18-47-02.690530__19',
 '2025-03-27_18-47-02.730714__25',
 '2025-03-27_18-47-02.619700__0',
 '2025-03-27_18-47-02.627604__10',
 '2025-03-27_18-47-02.738661__26',
 '2025-03-27_18-47-02.620631__2',
 '2025-03-27_18-47-02.722518__24',
 '2025-03-27_18-47-02.711009__23',
 '2025-03-27_18-47-02.705389__

In [337]:
dados = []
for model in lista:    
    with open(f'{path}/{model}/args.json', 'r') as file:
        args = json.load(file)
        
    n_reflect = args['num_reflections']
    tao = args['tao']
    colision_flag = args['colision']
    agent = PsAgent.load(f'{path}/{model}')
    matrix = agent.h_matrix
    if colision_flag:
        for state in [0,1]:
            for colision in [0,1]:
                for T in range(0,2000):
                    observation = [state,T,colision]
                    percept = agent.percept_preprocess(observation)

                    h_vector = matrix[:, percept]
                    sum_ = np.sum(h_vector)
                    probs = h_vector/sum_

                    nao_troca = probs[0]
                    troca = probs[1]

                    _ = {
                        'model':model,
                        'reflections':args['num_reflections'],
                        'T':T/tao,
                        'state':state,
                        'colision_flag':colision_flag,
                        'colision':colision,
                        'troca':troca,
                        'nao_troca':nao_troca,
                    }

                    dados.append(_)
    else:
        for state in [0,1]:
            for T in range(0,2000):
                observation = [state,T]
                percept = agent.percept_preprocess(observation)

                h_vector = matrix[:, percept]
                sum_ = np.sum(h_vector)
                probs = h_vector/sum_

                nao_troca = probs[0]
                troca = probs[1]

                _ = {
                    'model':model,
                    'reflections':args['num_reflections'],
                    'T':T/tao,
                    'state':state,
                    'colision_flag':colision_flag,
                    'colision':np.nan,
                    'troca':troca,
                    'nao_troca':nao_troca,
                }

                dados.append(_)

# Sem colisão

In [338]:
db = pd.DataFrame(dados)
db

,model,reflections,T,state,colision_flag,colision,troca,nao_troca
0,2025-03-27_18-47-02.622236__5,1,0.0000,0,1,0,0.004371,0.995629
1,2025-03-27_18-47-02.622236__5,1,0.0001,0,1,0,0.000560,0.999440
2,2025-03-27_18-47-02.622236__5,1,0.0002,0,1,0,0.012989,0.987011
3,2025-03-27_18-47-02.622236__5,1,0.0003,0,1,0,0.019372,0.980628
4,2025-03-27_18-47-02.622236__5,1,0.0004,0,1,0,0.010619,0.989381
...,...,...,...,...,...,...,...,...
255995,2025-03-27_18-47-02.702660__20,1,0.1995,1,1,1,0.001000,0.999000
255996,2025-03-27_18-47-02.702660__20,1,0.1996,1,1,1,0.001000,0.999000
255997,2025-03-27_18-47-02.702660__20,1,0.1997,1,1,1,0.001000,0.999000
255998,2025-03-27_18-47-02.702660__20,1,0.1998,1,1,1,0.001000,0.999000


In [339]:
db = pd.DataFrame(dados)
#db = db[~db['colision_flag'].apply(bool)]
db = db.sort_values(['model','state','T']).reset_index(drop = True)

db['prod_nao_troca'] = db.groupby(['model','state'])['nao_troca'].cumprod().shift(1)
db['P_T_troca'] = db['prod_nao_troca']*db['troca']
db['sum_P_T_troca'] = db.groupby(['model','state'])['P_T_troca'].cumsum().shift(1)

db_mean = db.groupby(['reflections','T','state'])[
    ['troca', 'P_T_troca', 'sum_P_T_troca']
].mean().reset_index()

db_mean.head()

,reflections,T,state,troca,P_T_troca,sum_P_T_troca
0,1,0.0000,0,0.004139,0.000734,0.499959
1,1,0.0000,1,0.048986,0.048941,0.496588
2,1,0.0001,0,0.002692,0.002671,0.003281
3,1,0.0001,1,0.084912,0.078550,0.097907
4,1,0.0002,0,0.003673,0.003622,0.009452


In [340]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('troca').axis(format = '%'),
    row = 'state:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [341]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('P_T_troca').axis(format = '%'),
    row = 'state:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [342]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    color = 'state:N',
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

# Com colisão

In [363]:
db = pd.DataFrame(dados)

#db = db[db['model'].isin(["2025-03-27_13-20-00.577568__15","2025-03-27_13-20-00.576201__12", "2025-03-27_13-20-00.574117__9"])]

db = db[db['colision_flag'].apply(bool)]
db = db.sort_values(['model','state','colision','T']).reset_index(drop = True)
db['prod_nao_troca'] = db.groupby(['model','state','colision'])['nao_troca'].cumprod().shift(1)
db['P_T_troca'] = db['prod_nao_troca']*db['troca']
db['sum_P_T_troca'] = db.groupby(['model','state','colision'])['P_T_troca'].cumsum().shift(1)

db_mean = db[db['T'].between(0.0001,0.02)].groupby(['reflections','T','state','colision'])[
    ['troca', 'P_T_troca', 'sum_P_T_troca']
].mean().reset_index()

db_mean.head()

,reflections,T,state,colision,troca,P_T_troca,sum_P_T_troca
0,1,0.0001,0,0,0.003696,0.003678,1.672019e-08
1,1,0.0001,0,1,0.001687,0.001685,4.730218e-13
2,1,0.0001,1,0,0.000055,0.000055,2.355605e-13
3,1,0.0001,1,1,0.169769,0.157065,8.208953e-04
4,1,0.0002,0,0,0.005399,0.005348,3.677947e-03


In [364]:
alt.Chart(db_mean).mark_line(opacity = 0.7).encode(
    x = 'T',
    y = alt.Y('troca').axis(format = '%'),
    row = 'colision:N',
    color = 'state:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [365]:
alt.Chart(db_mean).mark_line(opacity = 0.7).encode(
    x = 'T',
    y = alt.Y('P_T_troca').axis(format = '%'),
    row = 'colision:N',
    color = 'state:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [366]:
alt.Chart(db_mean).mark_line(opacity = 0.7).encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    row = 'colision:N',
    color = 'state:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)